In [9]:
#Please install necessary packages before running the code.
#install pywebio package
#pip install -U pywebio #Uncomment this line to install the package and execute
#install archive files that belong to pywebio
# pip install -U https://code.aliyun.com/wang0618/pywebio/repository/archive.zip #Uncomment this line to install the package and execute

In [11]:
# Importing needed libraries for generating data and render our plots inline

import pandas as pd #importing panda in order to deal with data
import math #importing math library in order to deal with data

import re #We will deal with regular expressions
df = pd.read_excel(r'cstcourses_grades.xlsx') #Reading the file cstcourses_grade.xlsx; make sure you have this file with the same folder where you have the python code file

In [13]:
#list of courses we have
def list_courses():
#    courses=set(df['Catalog'])
    courses=set(list(zip(df['Catalog'],df['Label'])))
    new_courses=[]
    for cat,lab in courses:
        new_courses.append(str(int(cat)) +" " + lab)
    
    return new_courses

# Arranging the grade with numerical scale
def get_grade(x):
    if x==4:
        return 'A'
    elif x==3:
        return 'B'
    elif x==2:
        return 'C'
    else:
        return 'D'
# Assuming that the average grade that was generated from data is ranged from A-D
def get_numberofhours(g):
    grade_hours={'A':'''You are more likely to be fine at this class. You still need to study at least 3 hours per week for a grade A.''',
                 'B':''' This class will require from you some attention. You need to study at least 5 hours per week for a grade B.''',
                 'C':''' This class is challenging. However, studying at least 8 hours per week will save you and hope for a grade C.''',
                 'D':'''You like to be challenged!!! This class required your full attention. At least 10 hours per week is required for this class for a grade D.'''}
    result=grade_hours[g]
    return result

In [14]:
reg=r'[0-9]{4}'  # selecting only numbers 
grade_numbers={'A':4,'B':3,'C':2,'D':1} # scalling grades with numbers from 1-4
courses_performance={}
courses=list_courses()
# This is the code where the application is going to calculate the average grades of each classbased on data given 
for element in courses:
    res=re.findall(reg,element)
    item=int(res[0])
    cond=(df['Catalog']==item)
    subset=df[cond]
    my_sum=0
    my_avg=0
    for grade in subset['Grade']:
        my_sum +=grade_numbers[grade]
    my_avg=int(my_sum/len(subset['Grade']))
    courses_performance[item]={}
    courses_performance[item]['Grade']=get_grade(my_avg)
    courses_performance[item]['Notes']=get_numberofhours(courses_performance[item]['Grade'])
# print(courses_performance)


In [15]:
# we will use this function to check if a grade exist in a given data
def check_existant(c):
    if c not in courses_performance.keys():
        return False
    return True

In [16]:
# importing needed libraries 
# To get package from PyWebio
from pywebio import *
from pywebio.input import *
from pywebio.output import *

# To handle the browser 
from flask import Flask
from pywebio.platform.flask import webio_view
from pywebio import STATIC_PATH


app = Flask(__name__)

def check_classes():
    
    put_image('https://upload.wikimedia.org/wikipedia/en/3/30/City_Tech.png', width='100px') # formerly 350px
    put_success('📚 Welcome to StudyTime! This pseudo_app suggests study_hours for City Tech CST classes.')
    #put_success('Welcome to StudyTime! This pseudo_app is mainly handling CST catgories classes.')
    
    # Get data from the user
    student_info = input_group("Required information:", [
        input('Input your name', name='student_name', required=True),
        select('Which class do you want to inquire about?', options=list_courses(), name='course_code_label', required=True),
        radio('Which grade you are aiming for?', options=['A', 'B', 'C', 'D'], inline=True, name='student_grade', required=True), 
        input('How many hours you study for this class per week in average?', name='student_hours', type=NUMBER, required=True)
    ])
    
    
    reg=r'[0-9]{4}'
    res=re.findall(reg, student_info['course_code_label'])
    course_code=int(res[0])

    top_status = {'A': 3, 'B': 5,
                  'C': 8, 'D': 10}
    grade_control={'A': 4, 'B': 3,
                  'C': 2, 'D': 1}
                  
    class_dificulty=courses_performance[course_code]['Grade']

    # Check if student aims to have the same average grade that was obtained from data
    if student_info['student_grade'] == class_dificulty:
        if student_info['student_hours'] >= top_status[class_dificulty]:

            # Suggesting number of hours that student needs to study
            put_text(f"Since you are aiming for {student_info['student_grade']}, and the class average grade from the previous semesters was also {class_dificulty}, with the numbers of hours you specified, you are safe.")
        else:
            # Suggesting number of hours that student needs to study
            put_text('''You need to work more hours per week to pass this class with the grade you want. Here is more information:''', courses_performance[course_code]['Notes'])
        
    else:
        if grade_control[student_info['student_grade']] < grade_control[class_dificulty]:
            # Suggesting number of hours that student needs to study
            put_text(f"This class's average grade is {class_dificulty}. So, you should aim for at least {class_dificulty} or higher.")
        else:
            extra_hours=round(top_status[class_dificulty]*(1.3**(grade_control[class_dificulty])))
            if student_info['student_hours'] < extra_hours:
                # Suggesting number of hours that student needs to study
                put_text('''Your study hours seems to be less than the average number of hours required to get your desired grade: You need at least''',extra_hours, 'hours.')
                put_text('''Here is more information about your current class:''', courses_performance[course_code]['Notes'])
            else:
                # Suggesting number of hours that student needs to study
                put_text('With the hours you specified you are more likely going to get your desired grade. Good Luck!!! 🍀')

    put_markdown('Thank you for using StudyTime, %s 😊 \n\n[Generate another suggestion](http://localhost/) ' % student_info['student_name']) 
    
    textarea('How do you think this tool may be improved?', rows=3) 
    toast('Thank you for your input!')
    
app.add_url_rule('/', 'webio_view', webio_view(check_classes), methods=['GET','POST','OPTIONS'])
app.run(host="localhost", port=80)
if __name__ == '__main__':
    check_classes()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:80/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Aug/2021 22:15:10] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:10] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:10] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:10] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:10] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:11] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:12] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:12] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:13] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:13] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:15] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:15] "GET /?app=index HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 22:15:17] "GET 

In [ ]:
# Below are tests for parts of the code
#
#
